In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from tqdm import tqdm
from PIL import Image
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
IMG_SIZE = 256
BATCH_SIZE = 64
EPOCHS = 100
LR = 0.001


train_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.0)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

val_transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])


In [4]:
train_dir = "/content/drive/MyDrive/archive/chest_xray/train"
val_dir   = "/content/drive/MyDrive/archive/chest_xray/val"
test_dir  = "/content/drive/MyDrive/archive/chest_xray/test"

train_dataset = datasets.ImageFolder(root=train_dir, transform=train_transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=val_transform)

#train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
#val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

from torch.utils.data import DataLoader, WeightedRandomSampler
import numpy as np

targets = [sample[1] for sample in train_dataset.samples]  # class indices
class_sample_counts = np.bincount(targets)
print("Samples per class:", class_sample_counts)


# Compute weights: inverse of class frequency
class_weights = 1.0 / class_sample_counts
print("Class weights:", class_weights)

# Assign weight for each sample in the dataset
sample_weights = [class_weights[t] for t in targets]

# Create sampler
sampler = WeightedRandomSampler(
    weights=sample_weights,
    num_samples=len(sample_weights),  # draw same number as dataset size
    replacement=True
)

# create DataLoader with sampler instead of shuffle
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    sampler=sampler
)

val_loader = DataLoader(
    val_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False
)


class_names = train_dataset.classes
print("Classes:", class_names)

Samples per class: [1183 3873]
Class weights: [0.00084531 0.0002582 ]
Classes: ['NORMAL', 'PNEUMONIA']


In [ ]:
'''
class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(2)
        )
    def forward(self, x):
        return self.block(x)

class SmallCNN64(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            ConvBlock(1, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
            ConvBlock(128, 256),
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(256 * (IMG_SIZE // 16) * (IMG_SIZE // 16), 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)
'''

class ConvBlock(nn.Module):
    def __init__(self, in_ch, out_ch, dropout=0.3):
        super().__init__()
        self.block = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.SiLU(inplace=True),
            nn.Conv2d(out_ch, out_ch, 3, padding=1),
            nn.BatchNorm2d(out_ch),
            nn.SiLU(inplace=True),
            nn.Dropout2d(dropout),
            nn.MaxPool2d(2)
        )

    def forward(self, x):
        return self.block(x)

class SmallCNN64(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            ConvBlock(1, 32),
            ConvBlock(32, 64),
            ConvBlock(64, 128),
            ConvBlock(128, 256),
        )
        self.classifier = nn.Sequential(
            nn.AdaptiveAvgPool2d((1,1)),   # GAP
            nn.Flatten(),
            nn.Linear(256, 512),
            nn.SiLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        return self.classifier(x)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SmallCNN64(num_classes=len(class_names)).to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scaler = torch.amp.GradScaler("cuda", enabled=torch.cuda.is_available())


def train_model(model, train_loader, val_loader, epochs=EPOCHS):
    train_losses, val_losses, val_accs = [], [], []

    for epoch in range(epochs):
        model.train()
        running_loss = 0.0

        for images, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.amp.autocast("cuda", enabled=torch.cuda.is_available()):
                outputs = model(images)
                loss = criterion(outputs, labels)

            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

            running_loss += loss.item()

        avg_train_loss = running_loss / len(train_loader)
        train_losses.append(avg_train_loss)

        # Validation
        model.eval()
        correct, total, val_loss = 0, 0, 0.0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                total += labels.size(0)
                correct += predicted.eq(labels).sum().item()

        avg_val_loss = val_loss / len(val_loader)
        val_losses.append(avg_val_loss)
        val_acc = 100 * correct / total
        val_accs.append(val_acc)

        print(f"Epoch {epoch+1}: Train Loss={avg_train_loss:.4f}, "
              f"Val Loss={avg_val_loss:.4f}, Val Acc={val_acc:.2f}%")

    return train_losses, val_losses, val_accs


import matplotlib.pyplot as plt

def plot_training(train_losses, val_losses, val_accs):
    epochs = range(1, len(train_losses) + 1)

    plt.figure(figsize=(14,5))

    # Plot Loss
    plt.subplot(1,2,1)
    plt.plot(epochs, train_losses, 'b-', label="Training Loss")
    plt.plot(epochs, val_losses, 'r-', label="Validation Loss")
    plt.xlabel("Epochs")
    plt.ylabel("Loss")
    plt.title("Training vs Validation Loss")
    plt.legend()
    plt.grid(True)

    # Plot Accuracy
    plt.subplot(1,2,2)
    plt.plot(epochs, val_accs, 'g-', label="Validation Accuracy")
    plt.xlabel("Epochs")
    plt.ylabel("Accuracy (%)")
    plt.title("Validation Accuracy")
    plt.legend()
    plt.grid(True)

    plt.show()

# Call it after training



train_losses, val_losses, val_accs = train_model(model, train_loader, val_loader, epochs=EPOCHS)
plot_training(train_losses, val_losses, val_accs)

def predict_image(image_path, model, class_names):
    model.eval()
    img = Image.open(image_path).convert("L")
    transform = val_transform
    img_t = transform(img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = model(img_t)
        _, predicted = outputs.max(1)

    return class_names[predicted.item()]

TP = 0
FP = 0
TN = 0
FN = 0
'''
for filename in os.listdir(test_dir):
    test_img = os.path.join(test_dir, filename)
    print("Test Image:", test_img)
    predicted = predict_image(test_img, model, class_names)
    print("Predicted:", predicted)
    if "N" in filename:
      if predicted == "NORMAL":
        TN += 1
      else:
        FP += 1
    if "P" in filename:
      if predicted == "PNEUMONIA":
        TP += 1
      else:
        FN += 1
'''
count = 1
for filename in os.listdir(test_dir):
    test_img = os.path.join(test_dir, filename)
    print("Test Image:", test_img)
    predicted = predict_image(test_img, model, class_names)
    print("Predicted:", predicted)
    if count < 51:
      if predicted == "NORMAL":
        TN += 1
      else:
        FP += 1
    if count >= 51:
      if predicted == "PNEUMONIA":
        TP += 1
      else:
        FN += 1
    count += 1

print("TP:", TP)
print("FP:", FP)
print("TN:", TN)
print("FN:", FN)



Epoch 1/100: 100%|██████████| 79/79 [18:17<00:00, 13.90s/it]


Epoch 1: Train Loss=0.4804, Val Loss=0.4651, Val Acc=79.00%


Epoch 2/100: 100%|██████████| 79/79 [09:54<00:00,  7.53s/it]


Epoch 2: Train Loss=0.3965, Val Loss=0.4188, Val Acc=80.67%


Epoch 3/100: 100%|██████████| 79/79 [06:00<00:00,  4.57s/it]


...

Epoch 37/100: 100%|██████████| 79/79 [02:06<00:00,  1.60s/it]


Epoch 37: Train Loss=0.1541, Val Loss=0.1632, Val Acc=92.67%


Epoch 38/100: 100%|██████████| 79/79 [02:07<00:00,  1.62s/it]


Epoch 38: Train Loss=0.1446, Val Loss=0.2344, Val Acc=90.50%


Epoch 39/100:  58%|█████▊    | 46/79 [01:15<00:55,  1.67s/it]

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

conf_matrix = np.array([[TN, FP],
                        [FN, TP]])

# Labels
labels = ["Negative", "Positive"]
classes = ["NORMAL","PNEUMONIA"]
# Plot with seaborn
plt.figure(figsize=(6,5))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues",
            xticklabels=classes, yticklabels=labels)


plt.title("Confusion Matrix")
plt.show()


In [ ]:
import math

accuracy  = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) else 0
recall    = TP / (TP + FN) if (TP + FN) else 0
specificity = TN / (TN + FP) if (TN + FP) else 0
fpr       = FP / (FP + TN) if (FP + TN) else 0
fnr       = FN / (FN + TP) if (FN + TP) else 0
npv       = TN / (TN + FN) if (TN + FN) else 0
f1        = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
balanced_accuracy = (recall + specificity) / 2
mcc_num   = (TP*TN - FP*FN)
mcc_den   = math.sqrt((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))
mcc       = mcc_num / mcc_den if mcc_den else 0

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("Specificity:", specificity)
print("F1 Score:", f1)
print("Balanced Accuracy:", balanced_accuracy)
print("MCC:", mcc)
